<a href="https://colab.research.google.com/github/fhhjhhg/AI-Notebooks/blob/main/MindsCloud_Mind_via_AssistantsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install minds_sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.2 MB/s eta 0:00:00
  Created wheel for minds_sdk: filename=minds_sdk-1.0.4-py3-none-any.whl size=7081 sha256=20ffbcbe6774a00a2d138008ddd872585b0715e63ad000629be84bedfda3a93f
  Stored in directory: /root/.cache/pip/wheels/83/20/3f/912673b73ad8b64499ac905b5497ec7eadf40cb5ccd3cc5cf4
Successfully built minds_sdk
  Attempting uninstall: pydantic-

In [ ]:
from google.colab import userdata
from datetime import datetime
from openai import OpenAI
from minds.client import Client
from minds.datasources import DatabaseConfig

client = Client(userdata.get('minds_api_key'))

postgres_config = DatabaseConfig(
    name='my_data',
    description='House sales data',
    engine='postgres',
    connection_data={
        'user': 'demo_user',
        'password': 'demo_password',
        'host': 'samples.mindsdb.com',
        'port': '5432',
        'database': 'demo',
        'schema': 'demo_data'
    },
    tables=['house_sales']
)

datasource = client.datasources.create(postgres_config, replace=True)

mind = client.minds.create(name='my_mind', datasources=[datasource], replace=True)

print(f"{mind.name} was created successfully. You can now use this Mind using the OpenAI-compatible API, see docs for next steps.")

my_mind was created successfully. You can now use this Mind using the OpenAI-compatible API, see docs for next steps.


In [ ]:
# point the Openai SDK to the Minds Cloud
client = OpenAI(
    api_key=userdata.get('minds_api_key'),
    base_url='https://llm.mdb.ai/'
)

thread = client.beta.threads.create()

In [ ]:
# Add a message to a thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How many 2-bedroom houses are on the market?"
)

# run the message on the mind that you created
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=mind.name
)

if run.status == 'completed':
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    print(messages.data[1].role + ': ' + messages.data[1].content[0].text.value)
    print(messages.data[0].role + ': ' + messages.data[0].content[0].text.value)
else:
    print(run.status)

user: How many 2-bedroom houses are on the market?
assistant: There are 99 two-bedroom houses currently on the market.

If you have any other questions or need further assistance, feel free to ask!


In [ ]:
# Add a message to a thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Which of them is the most expensive?"
)

# run the message on the mind that you created
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=mind.name
)

if run.status == 'completed':
    messages = client.beta.threads.messages.list(
      thread_id=thread.id
    )
    print(messages.data[1].role + ': ' + messages.data[1].content[0].text.value)
    print(messages.data[0].role + ': ' + messages.data[0].content[0].text.value)
else:
    print(run.status)

user: Which of them is the most expensive?
assistant: The most expensive 2-bedroom house currently on the market is priced at $510,712. Here are the details:

- **Sale Date:** 2019-09-30
- **Price (ma):** $510,712
- **Type:** House
- **Bedrooms:** 2
- **Created At:** 2007-02-06 05:27:08.378116

If you have any other questions or need further assistance, feel free to ask!


In [ ]:
while True:
    question = input("Ask a question (type 'exit' to quit): ")
    if question.lower() == "exit":
        break

    if question:
        message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=question
        )

        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=mind.name
        )

        if run.status == 'completed':
            messages = client.beta.threads.messages.list(
            thread_id=thread.id
            )
            print(messages.data[0].content[0].text.value)
        else:
            print(run.status)

client.beta.threads.delete(thread.id)

Ask a question (type 'exit' to quit): How many houses were sold in 2008?
A total of 28 houses were sold in 2008.

If you have any other questions or need further assistance, feel free to ask!
Ask a question (type 'exit' to quit): What was the price and details of the most expensive one?
The most expensive house sold in 2008 is priced at $764,025. Here are the details:

- **Sale Date:** 2008-12-31
- **Price (ma):** $764,025
- **Type:** House
- **Bedrooms:** 5
- **Created At:** 2007-02-24 12:05:51.632879

If you have any other questions or need further assistance, feel free to ask!
Ask a question (type 'exit' to quit): exit


ThreadDeleted(id='mdb_thread_10beee89a04640999a7ddc98b6f393f6', deleted=True, object='thread.deleted')